In [13]:
import pandas as pd
import numpy as np
import ast
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
#I used KNN and
def clean_text(text):
    """Remove special characters, convert to lowercase, remove stopwords, and lemmatize the text."""
    text = re.sub(r"[^\w\s]", "", text.lower())
    words = text.split()
    cleaned_text = " ".join(lemmatizer.lemmatize(word) for word in words if word not in stop_words)
    return cleaned_text

def load_data(file_path):
    """Load the movie dataset from a CSV file."""
    df = pd.read_csv(file_path, low_memory=False)
    return df

def check_columns(df):
    """Check if the necessary columns are present in the dataset."""
    required_columns = ["title", "overview", "genres", "original_language"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Dataset must contain '{col}' column.")
    return True

def preprocess_data(df):
    """Fill missing overviews with empty strings and extract genres."""
    df["overview"] = df["overview"].fillna("")
    def extract_genres(genre_str):
        try:
            genres_list = ast.literal_eval(genre_str)
            return " ".join([genre["name"] for genre in genres_list])
        except (ValueError, SyntaxError):
            return ""
    df["genres"] = df["genres"].apply(extract_genres)
    df["overview"] = df["overview"].apply(clean_text)
    df["genres"] = df["genres"].apply(clean_text)
    return df

def filter_by_genre(df, user_input):
    """Filter movies based on user-specified genre keywords."""
    keywords = clean_text(user_input).split()
    if not keywords:
        return df
    pattern = '|'.join(keywords)
    filtered_df = df[df['genres'].str.contains(pattern, case=False)]
    return filtered_df if not filtered_df.empty else df

def create_content(df):
    """Combine genres and overview into a single content string."""
    df["content"] = df["genres"] + " " + df["overview"]
    return df

def compute_tfidf(df):
    """Transform text data into TF-IDF vectors."""
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=0.01)
    tfidf_matrix = vectorizer.fit_transform(df["content"])
    return tfidf_matrix, vectorizer

def extract_language(user_input, language_mapping):
    """Extract language code from user input using a predefined mapping."""
    cleaned_input = clean_text(user_input)
    for word in cleaned_input.split():
        if word in language_mapping:
            return language_mapping[word], word
    return None, ''

def recommend_movies(user_input, df, tfidf_matrix, vectorizer, top_n=5):
    """Find the most similar movies based on user input description using KNN."""
    language_mapping = {
        'english': 'en',
        'french': 'fr',
        'spanish': 'es',
        'german': 'de',
        'italian': 'it',
        'japanese': 'ja',
        'chinese': 'zh',
        'korean': 'ko',
        'hindi': 'hi',
        'russian': 'ru',
        'dutch': 'nl',
        'portuguese': 'pt',
        'swedish': 'sv',
        'turkish': 'tr',
        'arabic': 'ar',
    }
    language_code, lang_keyword = extract_language(user_input, language_mapping)
    # Remove the language keyword from the user input for genre/keyword processing
    user_input_modified = clean_text(user_input)
    if lang_keyword:
        user_input_modified = user_input_modified.replace(lang_keyword, '').strip()
    # Filter by genre using the modified user input
    filtered_df = filter_by_genre(df, user_input_modified)
    # Further filter by language if detected
    if language_code:
        filtered_df = filtered_df[filtered_df['original_language'] == language_code]

    if filtered_df.empty:
        return pd.DataFrame()
    # Transform the modified user input for similarity comparison
    user_vector = vectorizer.transform([user_input_modified])
    # Fit KNN on the filtered data
    knn = NearestNeighbors(n_neighbors=top_n, metric='cosine')
    knn.fit(tfidf_matrix[filtered_df.index])
    distances, indices = knn.kneighbors(user_vector)
    # Prepare recommendations
    recommendations = filtered_df.iloc[indices.flatten()][["title", "genres", "overview", "original_language"]].copy()
    recommendations['similarity_score'] = 1 - distances.flatten()
    return recommendations.head(top_n)

def main(file_path, user_input):
    """Run the movie recommendation system."""
    df = load_data(file_path)
    check_columns(df)
    df = preprocess_data(df)
    df = create_content(df)
    tfidf_matrix, vectorizer = compute_tfidf(df)
    recommendations = recommend_movies(user_input, df, tfidf_matrix, vectorizer)
    print("\nTop Movie Recommendations:")
    if recommendations.empty:
        print("No recommendations found.")
    else:
        print(recommendations.to_string(index=False))

if __name__ == "__main__":
    file_path = "/content/movies_metadata.csv"
    user_input = input("Describe the type of movies you like: ")
    main(file_path, user_input)

# My code uses Natural Language Processing (NLP) techniques to clean and preprocess text data before
# applying a K-Nearest Neighbors (KNN) model for movie recommendations. I used these 2 in combination because
# using onlt TF-IDF was not giving me very accurate results. The code begins by loading the dataset and
# verifying essential columns like title, overview, genres, and original language. The text preprocessing
# involves removing special characters, converting text to lowercase, and eliminating stopwords using the NLTK
# stopwords list. It also applies lemmatization through the WordNet Lemmatizer to reduce words to their base
# forms, ensuring better text consistency. Missing overviews are filled with empty strings, and genres are
# extracted using ast.literal_eval to convert string representations of lists into actual lists. The cleaned
# text data is combined into a single content feature, which is then transformed into a TF-IDF matrix to
# represent text numerically. The user input undergoes similar text cleaning and is checked for language
# keywords using a predefined mapping. The dataset is filtered based on the detected language and relevant
# genres before applying a KNN model trained on the TF-IDF matrix to find the most similar movies based on
# cosine similarity. The system then returns the top recommendations, including movie titles, genres,
# overviews, and similarity scores.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Describe the type of movies you like: french comedies

Top Movie Recommendations:
                  title genres overview original_language  similarity_score
          Summer Nights comedy                         fr               1.0
              Le Cactus comedy                         fr               1.0
 Le Nouveau Jean-Claude comedy                         fr               1.0
La stratégie de l'échec comedy                         fr               1.0
      Le maître d'école comedy                         fr               1.0
